In [62]:
import pandas as pd
from collections import Counter
import re
import math
df = pd.read_csv("./Data/2019_TFIDF_DBSCAN_CV_eps8_mp20.csv", encoding='utf-8-sig')

In [63]:
df = df.sort_values(by='labels', ascending=True)
df.tail()

,Unnamed: 0,date,press,category,headline,url,text,img,N_list,topics,labels,cnt
3099,124744,2019-06-26,동아일보,"국제,사회",강제철거 반나절만에 재설치… 광화문 ‘천막 전쟁’,http://www.donga.com/news/article/all/20190626...,"서울시, 우리공화당 텐트 없애자 조원진 “한개 치우면 두개 세울것”원래 있던 자리에...",['https://dimg.donga.com/wps/NEWS/IMAGE/2019/0...,"[['서울시', '우리공화당', '텐트', '조원진', '자리', '그늘', '서울...","철거, 천막, 공화당, 서울시, 강제, 집행, 지지자, 그늘, 행정, 광화문광장",60,61
3098,125841,2019-06-27,동아일보,"정치,사회",2년째 그 자리… 불법 흉물천막이 거리의 주인 됐다,http://www.donga.com/news/article/all/20190627...,청와대앞 인도에 줄줄이 설치 주민들 길막혀 차도로 걷기도세종문화회관 주변도 텐트 점...,['https://dimg.donga.com/wps/NEWS/IMAGE/2019/0...,"[['청와대', '인도', '줄', '주민', '길', '차도', '세종문화회관',...","천막, 철거, 우리공화당, 설치, 조원진, 서울시, 광화문광장, 계고장, 불법, 구청",60,61
3097,128155,2019-06-30,세계일보,"사회,국제",'광화문 천막 사태' 재발 우려…트럼프 떠난 뒤 다시 칠 가능성,http://www.segye.com/content/html/2019/06/30/2...,"우리공화당, 세 번째 설치땐 행정대집행 불가피서울시 ""근본대책 절실""경찰, 시설물보...",['//img.segye.com/content/image/2019/06/30/201...,"[['우리공화당', '행정', '집행', '서울시', '근본', '대책', '경찰'...","천막, 우리공화당, 광화문광장, 서울시, 행정, 집회, 집행, 철거, 경찰, 광장",60,61
3108,127260,2019-06-28,중앙일보,"국제,정치",조원진 “우리공화당 광화문 천막 다른곳으로 일시 이동”,https://news.joins.com/article/olink/23102219,우리공화당(구 대한애국당)이 도널드 트럼프 미국 대통령 방한을 하루 앞두고 애초 밝...,['https://pds.joins.com/news/component/htmlpho...,"[['우리공화당', '도널드 트럼프', '미국', '대통령', '방한', '하루',...","천막, 우리공화당, 광화문광장, 철거, 광화문, 광장, 설치, 텐트, 애국, 빌딩",60,61
3069,123860,2019-06-25,세계일보,"국제,사회",광화문광장 천막 첫 강제철거…비용 2억에 무단점거 변상금 별도,http://www.segye.com/content/html/2019/06/25/2...,광화문광장에 설치된 대한애국당(현 우리공화당) 천막 강제철거 비용은 2억원 선에 이...,['//img.segye.com/content/image/2019/06/25/201...,"[['광화문광장', '설치', '대한애국당', '천막', '강제', '철거', '비...","천막, 우리공화당, 광화문광장, 철거, 상금, 비용, 설치, 집행, 행정, 대한애국당",60,61


In [14]:
def npmi(a, b, n, z):
    x = math.log((n*z)/(a*b))
    y = math.log(n/z)
    return x / y

In [64]:
docs = []
for article in df['N_list']:
    doc = []
    com = []
    context = []
    merge_token = []
    merge_token2 = []

    # str -> list
    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        doc.append(arr)
        context += arr

    #2 PMI 적용
    for line in doc:
        com += list(zip(line, line[1:]))
    n = len(context)
    cnt = Counter(context)
    nmi = {}
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        if -1 < pmi < 1:
            nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = [key[0]+" "+key[1] for key, item in result if item >= 0.91]

    # 토큰을 합치는 내용
    for token in merge_token:
        for content in doc:
            for i in range(len(content)-1):
                cmp_str = ' '.join(content[i:i+2])
                if cmp_str == token:
                    del content[i:i+2]
                    content.insert(i, token)
    docs.append(doc)

In [65]:
df['N_list'] = docs
df[df['labels']==36]

,Unnamed: 0,date,press,category,headline,url,text,img,N_list,topics,labels,cnt
4419,20660,2019/01/31,서울신문,정치,"“북·미, 영변 핵시설 폐기에 우선 집중”",http://www.seoul.co.kr/news/newsView.php?id=20...,외교당국자 “美 상당한 상응조치 할 것” 북·미 정상회담을 위한 실무회담에서 북한의...,[],"[[외교, 당국자, 상당, 상응, 조치, 북, 미, 정상, 회담, 실무, 회담, 북...","영변, 상응, 폐기, 조치, 미국, 북한, 시설, 협상, 회담, 평양",36,12
4418,3978,2019/01/07,한겨레,정치,[김지석 칼럼] 개성공단·금강산관광 재개부터,http://www.hani.co.kr/arti/opinion/column/8773...,북한이 김정은 국무위원장의 신년사를 통해 밝힌 올해 국가전략은 두개의 구호로 집약된...,['http://img.hani.co.kr/imgdb/resize/2019/0107...,"[[북한, 김정은, 국무위원, 장, 신년사, 올해, 국가, 전략, 구호, 집약], ...","북한, 협상, 조처, 시설, 영변, 상응, 미국, 폐기, 제재, 비핵화",36,12
4417,16383,2019/01/25,동아일보,정치,웃으며 트럼프 친서 내보인 김정은… “한발 한발 함께” 추가조치 요구,http://news.donga.com/3/all/20190125/93854637/1,[北美 비핵화 협상]2차회담 높아지는 빅딜 기대감김정은 북한 국무위원장이 전례 없는...,['https://dimg.donga.com/wps/NEWS/IMAGE/2019/0...,"[[비핵화, 협상, 차, 회담, 빅딜, 기대감, 김정, 북한, 국무위원, 장, 전례...","북미, 폐기, 빅딜, 정상, 회담, 영변, 북한, 시설, 미국, 결단력",36,12
4409,33657,2019/02/25,국민일보,정치,"[단독] 北, 비건에게 “이번엔 영변 폐기까지” 통보",http://news.kmib.co.kr/article/view.asp?arcid=...,북한이 2차 북·미 정상회담 의제를 놓고 지난 6~8일 평양에서 미국 측과 실무협상...,[],"[[북한, 차, 북, 미, 정상, 회담, 의제, 평양, 미국, 측, 실무, 협상, ...","영변, 시설, 북한, 회담, 폐기, 동결, 정상, 협상, 알파, 플러스",36,12
4415,63667,2019/04/04,서울신문,정치,“대북 특사 파견하거나 원포인트 판문점 남북정상회담 바람직”,http://www.seoul.co.kr/news/newsView.php?id=20...,한국 정부가 한미정상회담 전 대북 특사를 파견하거나 원포인트 판문점 남북정상회담 개...,['//img.seoul.co.kr/img/upload/2019/04/04/SSI_...,"[[한국, 정부, 한미정상회담, 대북, 특사, 파견, 포인트 판문점, 남북, 정상 ...","북한, 한미정상회담, 회담, 협상, 미국, 제재, 영변, 시설, 폐기, 정상",36,12
4410,33294,2019/02/23,중앙일보,정치,"“북한, 영변폐기 합의 후 ‘살라미 전술’ 가능성”",https://news.joins.com/article/olink/22986164,김정은 북한 국무위원장이 2차 북미 정상회담에서 영변 핵시설 폐기와 추가적인 비핵화...,['https://pds.joins.com/news/component/htmlpho...,"[[김정, 북한, 국무, 위원장, 차, 북미, 정상 회담, 영변, 핵, 시설, 폐기...","영변, 시설, 폐기, 북한, 살라미, 회담, 정상, 협상, 조치, 합의",36,12
4414,30012,2019/02/19,국민일보,정치,"“北·美, 연락관 교환 신중 검토… 공식 외교관계 수립 향한 조치”",http://news.kmib.co.kr/article/view.asp?arcid=...,베트남 하노이에서 열릴 2차 북·미 정상회담을 앞두고 북한과 미국이 서로 연락관을 ...,[],"[[베트남 하노이, 차, 북 미, 정상 회담, 북한, 미국, 연락관, 교환, 방안,...","폐기, 영변, 시설, 검증, 회담, 북한, 정상, 미국, 양측, 비핵화",36,12
4413,12728,2019/01/20,세계일보,"정치,국제",스몰딜이냐 빅딜이냐… 北·美 6주간의 치열한 수싸움 돌입,http://www.segye.com/content/html/2019/01/20/2...,일정 윤곽 잡힌 ‘核 담판’ 전망 / 영변核·ICBM 폐기·대북제재 완화 등 다양한...,['//img.segye.com/content/image/2019/01/20/201...,"[[일정, 윤곽, 담판, 전망, 영변, 폐기, 대북, 제재, 완화, 다양 카드, 본...","북한, 미국, 회담, 대북, 제재, 스몰, 협상, 폐기, 영변, 부위원장",36,12
4412,25573,2019/02/12,국민일보,정치,"북 비핵화의 관문 영변 핵시설, 30년 넘게 지켜지지 않았던 폐기 약속",http://news.kmib.co.kr/article/view.asp?arcid=...,오는 27~28일 베트남 하노이에서 열리는 2차 북·미 정상회담의 성패는 영변 핵시...,[],"[[베트남, 하노이, 차, 북, 미, 정상 회담, 성패, 영변, 핵, 시설, 폐기,...","영변, 시설, 폐기, 북한, 회담, 협상, 실무, 정상, 약속, 상응",36,12
4416,28294,2019/02/15,서울신문,정치,폼페이오 ‘先비핵화’ 대신 “대북제재 완화” 공론화…영변 폐기 넘는 성과 거둘까,http://www.seoul.co.kr/news/newsView.php?id=20...,제재 완화 운 떼며 김정은에 비핵화 약속 이행 촉구 마이크 폼페이오 미국 국무장관...,['//img.seoul.co.kr/img/upload/2019/02/15/SSI_...,"[[제재 완화, 운, 김정은, 핵, 약속, 이행, 촉구, 마이크, 폼페이오, 미국,...","완화, 북한, 제재, 폼페이오, 영변, 폐기, 검증, 회담, 미국, 협상",36,12


In [1]:
import pandas as pd
from gensim.models import word2vec
from gensim.models import Word2Vec


for i in range(df['labels'].max()+1):
    word_list = []
    for article in df[df['labels'] == i]['N_list']:
        doc = []
        for line in article:
            doc += line
            word_list.append(doc)
    print(len(word_list))
    # to = df['topics']
    # to = topics.dropna()
    # fopic in topics:
    #   rd = topic.split(', ')
    #   r w in word:
    #     if(len(w) <= 1):
    #         word.remove(w)
    #   rd_list.append(word)
    # # l = Word2Vec.load('C:/Users/ghdtk/Project_test/ko.bin')
    # # l.wv.save_word2vec_format('C:/Users/ghdtk/Project_test/ko.bin.gz', binary = False)
    my_m = Word2Vec(word_list, window = 100, vector_size=200, min_count = 1, sg = 1,workers=4)


    # myel.intersect_word2vec_format('C:/Users/ghdtk/Project_test/ko.bin.gz', binary = False)

    wordtors = my_model.wv
    word_vectors.vocab.keys()
    wordtors_list = [word_vectors[v] for v in vocabs]

    # prword_vectors.similarity(w1='정부', w2='대통령'))
    prinrd_vectors.most_similar("김정은")
    break

NameError: name 'df' is not defined

In [156]:
import pandas as pd
from gensim.models import word2vec


word_list = []
for article in df[df['labels'] == 9]['N_list']: #번호에 해당 클러스터 번호입력
    doc = []
    for line in article:
        doc += line
        word_list.append(doc)
print(len(word_list))

my_model = Word2Vec(word_list, window = 100, min_count = 1, sg = 1,workers=4,vector_size=200)

word_vectors = my_model.wv
word_vectors.save('word_vec3')
#loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드


4822


TypeError: unhashable type: 'list'

In [110]:
from gensim.models import FastText

my_model_fast = FastText(word_list, window = 100, min_count = 1, sg = 1,vector_size=100)

word_vectors_fast = my_model_fast.wv
word_vectors_fast.save('word_fast3')
#loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드


In [155]:
from gensim.models import KeyedVectors

loaded_model=KeyedVectors.load("word_vec2")
loaded_model.most_similar("")

[('경협', 0.43442708253860474),
 ('김영철', 0.40458330512046814),
 ('고대', 0.3864108920097351),
 ('매파', 0.3814399540424347),
 ('노골적', 0.38012757897377014),
 ('통일전선부', 0.37808918952941895),
 ('조미수뇌', 0.3741137385368347),
 ('이발', 0.37381061911582947),
 ('차 북미', 0.36483627557754517),
 ('대톨령', 0.3632027804851532)]

In [154]:
loaded_model=KeyedVectors.load("word_fast2")
loaded_model.most_similar("정은")

[('이정은', 0.8745384812355042),
 ('페이지', 0.8439659476280212),
 ('병기', 0.8291098475456238),
 ('정착', 0.819145917892456),
 ('방침', 0.7895593643188477),
 ('디', 0.7649025917053223),
 ('상임', 0.7409874200820923),
 ('김정은', 0.7392602562904358),
 ('국가안전보장회', 0.7311857342720032),
 ('파악', 0.6985346078872681)]

In [98]:
import pandas as pd
from gensim.models import word2vec
from gensim.models import Word2Vec


word_list = []

for i in range(df['labels'].max()+1):
    for article in df[df['labels'] == i]['N_list']:
        doc = []
        for line in article:
            doc += line
            word_list.append(doc)
    
print(len(word_list))
my_model = Word2Vec(word_list, window = 100, min_count = 1, sg = 1,workers=4,vector_size=200)
word_vectors = my_model.wv
word_vectors.save('word_vec2')

81673


In [99]:
import pandas as pd
from gensim.models import word2vec


word_list = []
for article in df[df['labels'] == 36]['N_list']: #번호에 해당 클러스터 번호입력
    doc = []
    for line in article:
        doc += line
        word_list.append(doc)

print(len(word_list))
my_model_fast = FastText(word_list, window = 100, min_count = 1, sg = 1,vector_size=100)

word_vectors_fast = my_model_fast.wv
word_vectors_fast.save('word_fast2')


297
